# Premier exemple de classification: prédire le niveau de diplôme dans le recensement 

In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import time

In [ ]:
# Définir le répertoire courant
os.chdir('/home/onyxia/work/methodes_ensemblistes_notebooks')

# Vérification
print("Nouveau répertoire courant :", os.getcwd())

In [ ]:
# Fonction pour lire les fichiers Parquet
def read_parquet_data(local_path, var_name, force_reload=False):
    """
    Charge un fichier Parquet, avec possibilité de forcer le rechargement même si la variable est en mémoire.
    
    Args:
        local_path (str): Chemin du fichier Parquet.
        var_name (str): Nom de la variable à vérifier dans l'espace de noms global.
        force_reload (bool): Forcer le rechargement des données même si elles sont déjà en mémoire.
    
    Returns:
        pd.DataFrame: Données chargées depuis le fichier Parquet.
    """
    if var_name in globals() and not force_reload:
        print(f"Les données '{var_name}' sont déjà en mémoire. Utilisez force_reload=True pour les recharger.")
        return globals()[var_name]
    
    if os.path.exists(local_path):
        try:
            data = pq.read_table(local_path).to_pandas()
            print(f"Données Parquet chargées avec succès depuis : {local_path}")
            globals()[var_name] = data  # Stocker la variable dans l'espace global
            return data
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier Parquet : {e}")
    else:
        print(f"Le fichier n'existe pas : {local_path}")
    return None


# Fonction pour lire les fichiers CSV
def read_csv_data(local_path, var_name, force_reload=False):
    """
    Charge un fichier CSV, avec possibilité de forcer le rechargement même si la variable est en mémoire.
    
    Args:
        local_path (str): Chemin du fichier CSV.
        var_name (str): Nom de la variable à vérifier dans l'espace de noms global.
        force_reload (bool): Forcer le rechargement des données même si elles sont déjà en mémoire.
    
    Returns:
        pd.DataFrame: Données chargées depuis le fichier CSV.
    """
    if var_name in globals() and not force_reload:
        print(f"Les données '{var_name}' sont déjà en mémoire. Utilisez force_reload=True pour les recharger.")
        return globals()[var_name]
    
    if os.path.exists(local_path):
        try:
            data = pd.read_csv(local_path, sep=';')
            print(f"Données CSV chargées avec succès depuis : {local_path}")
            globals()[var_name] = data  # Stocker la variable dans l'espace global
            return data
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier CSV : {e}")
    else:
        print(f"Le fichier n'existe pas : {local_path}")
    return None

In [ ]:
# Charger les données si elles ne sont pas déjà en mémoire
data_census_individuals = read_parquet_data("data/data_census_individuals.parquet", "data_census_individuals", force_reload=True)

# Charger la documentation
doc_census_individuals = read_csv_data("documentation/doc_census_individuals.csv", "doc_census_individuals", force_reload=True)

In [ ]:
# Échantillonner les données (1/200)
data_sample = data_census_individuals.sample(frac=1/200, random_state=123)

In [ ]:
# Suppression des variables liées à l'âge (autres que AGED) et des variables non pertinentes ou avec trop de modalités (pour commencer)
columns_to_drop = [
    'AGER20', 'AGEREV', 'AGEREVQ', 'ANAI', 'TRIRIS', 'IRIS', 'DNAI',
    'DEPT', 'ARM', 'CANTVILLE', 'NUMMI', 'IPONDI'
]
data_clean = data_sample.drop(columns=columns_to_drop)

In [ ]:
# Voir la répartition des classes de diplôme
print(data_clean['DIPL'].isna().sum())  # Vérification des valeurs manquantes
print(data_clean['DIPL'].value_counts())  # Distribution des classes
print(pd.crosstab(data_clean['DIPL'], data_clean['CS1'], normalize='columns') * 100)


In [ ]:
# Séparer les données en features (X) et la variable cible (y : DIPL)
X = data_clean.drop(columns=['DIPL'])
y = data_clean['DIPL']

In [ ]:
# Encodage "one-hot" des colonnes catégoriques
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Identifier les colonnes catégoriques
categorical_cols = X.select_dtypes(include=['category']).columns

# Appliquer le One-Hot Encoding sur les colonnes catégoriques
X_encoded = encoder.fit_transform(X[categorical_cols])

# Convertir les colonnes encodées en DataFrame avec les noms des catégories
X_encoded = pd.DataFrame(
    X_encoded, 
    index=X.index, 
    columns=encoder.get_feature_names_out(categorical_cols)
)

# Sélectionner les colonnes numériques
numeric_cols = X.select_dtypes(exclude=['category'])

# Concaténer les colonnes numériques et encodées uniquement si les colonnes numériques ne sont pas vides
if not numeric_cols.empty:
    X = pd.concat([numeric_cols, X_encoded], axis=1)
else:
    X = X_encoded

# Vérification finale
print(f"Dimensions finales après encodage : X={X.shape}")
print("Aperçu de X :")
print(X.head())

# Vérifier les dimensions après division
print(f"Dimensions après division : X={X.shape}")

In [ ]:
# Dimensions avant et après l'encodage
print(f"Dimensions avant encodage : {X.shape[0]} lignes, {categorical_cols.shape[0]} colonnes catégoriques")

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
# Calculer les poids des observations en fonction de l'inverse des fréquences dans y_train
class_weights = {cls: 1 / count for cls, count in zip(*np.unique(y_train, return_counts=True))}
class_weights_normalized = {cls: weight / sum(class_weights.values()) for cls, weight in class_weights.items()}

In [ ]:
# Modèle Random Forest pour classification
rf_model = RandomForestClassifier(
    n_estimators=500,
    max_features='sqrt',
    min_samples_leaf=5,
    class_weight=class_weights_normalized,
    random_state=123
)

In [ ]:
# Entraînement du modèle
import time
start_time = time.time()
rf_model.fit(X_train, y_train)
elapsed_time = time.time() - start_time
print(f"Temps d'exécution du modèle Random Forest : {elapsed_time:.2f} secondes")

In [ ]:
# Prédictions sur les données de test
y_pred = rf_model.predict(X_test)

In [ ]:
# Évaluer la performance avec une matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred, labels=rf_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=rf_model.classes_)
disp.plot(cmap="Blues")
plt.title("Matrice de confusion pour la prédiction du niveau de diplôme")
plt.show()

In [ ]:
# Rapport de classification
print(classification_report(y_test, y_pred))

In [ ]:
# Importance des variables (Mean Decrease in Impurity)
importance_df = pd.DataFrame({
    'Variable': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

In [ ]:
# Rassembler l'importance des modalités d'une même variable
def calculate_aggregated_importance(X, feature_importances):
    """
    Agrège l'importance des variables catégorielles encodées en colonnes multiples.

    Args:
        X (pd.DataFrame): Les données d'entraînement (features).
        feature_importances (array): Les importances des variables calculées par le modèle.

    Returns:
        pd.DataFrame: Importance des variables agrégées par nom de variable.
    """
    # Récupérer les noms des colonnes
    feature_names = X.columns
    
    # Initialiser un dictionnaire pour stocker les importances agrégées
    aggregated_importance = {}

    for col in feature_names:
        # Identifier les variables spécifiques pour regrouper correctement
        if col.startswith('STAT_CONJ_'):
            variable = 'STAT_CONJ'
        elif col.startswith('STATR_'):
            variable = 'STATR'
        else:
            # Utiliser le préfixe par défaut basé sur le premier segment avant "_"
            variable = col.split('_')[0]
        
        # Additionner les importances pour chaque variable "parent"
        if variable in aggregated_importance:
            aggregated_importance[variable] += feature_importances[feature_names.get_loc(col)]
        else:
            aggregated_importance[variable] = feature_importances[feature_names.get_loc(col)]

    # Convertir en DataFrame trié par importance
    aggregated_df = pd.DataFrame({
        'Variable': aggregated_importance.keys(),
        'Importance': aggregated_importance.values()
    }).sort_values(by='Importance', ascending=False)

    return aggregated_df

In [ ]:
# Appeler la fonction pour regrouper les importances
aggregated_importance_df = calculate_aggregated_importance(X, rf_model.feature_importances_)

# Afficher les résultats
print(aggregated_importance_df)

In [ ]:
# Charger les libellés des variables depuis le dictionnaire
doc_census_individuals_noms_variables = doc_census_individuals[['COD_VAR', 'LIB_VAR']].drop_duplicates()

In [ ]:
# Associer les libellés (descriptions) aux variables d'importance
importance_df_with_labels = aggregated_importance_df.merge(
    doc_census_individuals_noms_variables,
    left_on='Variable',
    right_on='COD_VAR',
    how='left'
)

In [ ]:
# Fonction pour visualiser des n variables les plus importantes
def plot_top_n_variables(importance_df, variable_col='Variable', importance_col='Importance', top_n=10):
    """
    Visualise les n variables les plus importantes à partir d'un DataFrame d'importance.

    Parameters:
    - importance_df (pd.DataFrame): Un DataFrame contenant les colonnes spécifiées.
    - variable_col (str): Le nom de la colonne contenant les noms des variables.
    - importance_col (str): Le nom de la colonne contenant les valeurs d'importance.
    - top_n (int): Le nombre de variables les plus importantes à afficher.
    """
    # Vérifier si les colonnes existent
    if variable_col not in importance_df.columns or importance_col not in importance_df.columns:
        raise ValueError(f"Les colonnes '{variable_col}' et/ou '{importance_col}' ne sont pas présentes dans le DataFrame.")

    # Trier par importance décroissante
    top_variables = importance_df.sort_values(by=importance_col, ascending=False).head(top_n)

    # Création du graphique
    plt.figure(figsize=(10, 8))
    plt.barh(top_variables[variable_col], top_variables[importance_col], color='steelblue')
    plt.xlabel("Importance")
    plt.ylabel("Variable")
    plt.title(f"Top {top_n} variables les plus importantes")
    plt.gca().invert_yaxis()  # Afficher les plus importantes en haut
    plt.show()

In [ ]:
# Visualisation des variables les plus importantes
plot_top_n_variables(
    importance_df = importance_df_with_labels,
    variable_col='LIB_VAR',
    importance_col='Importance',
    top_n=10
)